# Scraping and processing rain data
This notebook scraps Meteosuisse rainfall data from a third-party website holding a database

In [1]:
from lxml import html
import requests
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

from matplotlib import rc
rc('font',**{'family':'serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
plt.rcParams.update({'font.size': 22})

## Scraping the data

In [2]:
df = pd.DataFrame()

In [3]:
DayMth = np.array([[1, 31],[2,28],[3,31],[4,30],[5,31],[6,30],[7,31],[8,31],[9,30],[10,31],[11,30],[12,31]])
Data = np.zeros((1,7300))            

for year in np.arange(2000,2020):
    cnt = 0
    for month in np.arange(1,13):
        StrMth = str(month)
        if month < 10:
            StrMth = '0'+StrMth
    
        for day in np.arange(1,DayMth[month-1,1]+1):
                   StrDay = str(day)
                   if day < 10:
                       StrDay = '0'+StrDay  
                   page = requests.get('https://www.prevision-meteo.ch/climat/horaire/geneve-cointrin/'+str(year)+'-'+StrMth+'-'+StrDay)
                   tree = html.fromstring(page.content)
                   a = str(html.tostring(tree))
                   b1 = a[a.find('Precipitations totale'):-1]
                   b2 = b1[b1.find('min</td>')+6:-1]
                   b3 = b2[b2.find(';">')+3:b2.find(';">')+8]
                   if '<' in b3:
                       b3 = b3[0:b3.find('<')]
                   Data = float(b3)
                   s1 = pd.Series({'Year' : int(year),
                                   'Month' : int(month),
                                   'Day' : int(day),
                                   'Precipitation' : Data})
                   df = df.append([s1])


        print(str(year)+'-'+StrMth)


    





2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
2003-01
2003-02
2003-03
2003-04
2003-05
2003-06
2003-07
2003-08
2003-09
2003-10
2003-11
2003-12
2004-01
2004-02
2004-03
2004-04
2004-05


ConnectionError: HTTPSConnectionPool(host='www.prevision-meteo.ch', port=443): Max retries exceeded with url: /climat/horaire/geneve-cointrin/2004-06-05 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11f3b87b8>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [ ]:
df.to_csv(r'PrecipitationGVA_daily.csv', index = False)

## Processing the data

In [ ]:
df

In [ ]:
df[(df['Year']==2000) & (df['Month']==1)]['Precipitation'].sum()
df[(df['Year']==2000)]

In [ ]:
plt.figure(figsize=(14,6))
for yr in df['Year'].unique():
    plt.plot(np.arange(1,354),df[(df['Year']==yr)].sort_values(by='Precipitation',ascending=False)['Precipitation'])
plt.xlim([0,150])
plt.legend()
plt.ylim([0,70])
plt.show()



In [ ]:
DaysNoRain = pd.Series()
for yr in df['Year'].unique():
    DaysNoRain = DaysNoRain.append(pd.Series({yr : len(df[(df['Year']==yr) & (df['Precipitation']<1)])}),ignore_index=False)

plt.figure(figsize=(14,6))
plt.plot(DaysNoRain,'-o')
plt.ylabel('Number of dry days')
plt.grid('True')
plt.xticks(np.arange(2000, 2020, 1),rotation='vertical')
plt.show()

In [ ]:
dryPeriod = np.array([])
cnt = 0
dfDry = pd.DataFrame()
for yr in df['Year'].unique():
    a = df[df['Year']==yr]['Precipitation']
    for d in np.arange(0,len(a)):
        if a.iloc[d]<0.1001:
            cnt += 1
        if (a.iloc[d]>0.1001) & (cnt>0):
            dryPeriod = np.append(dryPeriod,cnt)
            dfDry = dfDry.append(pd.DataFrame([{'Year' : yr,'Dry Days' : cnt, 
                                               }],index=[1]))
            cnt = 0
dfDry = dfDry.reset_index()
dfDry = dfDry.drop(columns='index')
dfDry

In [ ]:
dfDry.groupby('Year').max()

In [ ]:

plt.figure(figsize=(14,6))
plt.plot(dfDry[dfDry['Dry Days']>7]['Year'],dfDry[dfDry['Dry Days']>7]['Dry Days'],'<',alpha=0.3)
plt.grid('True')
plt.xticks(np.arange(2000, 2020, 1),rotation='vertical')
plt.show()
